# Environment Setup for Azure AI Foundry Workshop

This notebook will guide you through setting up your environment for the Azure AI Foundry workshop.

## Prerequisites
- Python 3.8 or later
- Azure subscription with AI services access
- Basic Python knowledge

In [ ]:
# Install required packages (if not already installed):
!pip install azure-identity azure-ai-projects

## Azure Authentication Setup
First, we'll verify our Azure credentials and setup.

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ConnectionType
import os

# Initialize Azure credentials
try:
    credential = DefaultAzureCredential()
    print("✓ Successfully initialized DefaultAzureCredential")
except Exception as e:
    print(f"× Error initializing credentials: {str(e)}")

## Fix Azure CLI Authentication Issues

If you're seeing MSAL/NormalizedResponse errors, let's fix the Azure CLI authentication cache first:

In [ ]:
# Fix Azure CLI authentication cache corruption
import os
import shutil
from pathlib import Path
from dotenv import load_dotenv

# Load environment variables first
load_dotenv()

print("🔧 Fixing Azure CLI authentication...")

# Clear Azure CLI cache
azure_cache_dir = Path.home() / '.azure'
if azure_cache_dir.exists():
    try:
        # Clear specific cache files that cause issues
        cache_files = [
            azure_cache_dir / 'msal_http_cache',
            azure_cache_dir / 'msal_token_cache.json',
            azure_cache_dir / 'accessTokens.json'
        ]
        
        for cache_file in cache_files:
            if cache_file.exists():
                cache_file.unlink()
                print(f"✅ Removed {cache_file.name}")
        
        print("✅ Azure CLI cache cleared")
    except Exception as e:
        print(f"⚠️ Cache clear error: {e}")

# Get authentication details from environment
tenant_id = os.getenv("TENANT_ID")
subscription_id = os.getenv("AZURE_SUBSCRIPTION_ID")

print(f"🔑 Using Tenant: {tenant_id}")
print(f"📋 Using Subscription: {subscription_id}")

# Use device code authentication (most reliable)
!az login --use-device-code --tenant {tenant_id}
!az account set --subscription {subscription_id}

## Initialize Credentials (After Authentication Fix)

Now let's initialize the Azure credentials with the fixed authentication:

In [ ]:
# Initialize Azure credentials with fallback options
from azure.identity import DefaultAzureCredential, AzureCliCredential, InteractiveBrowserCredential, ChainedTokenCredential

# Load environment to get the correct tenant
from dotenv import load_dotenv
load_dotenv()

# Get the correct tenant ID from environment
correct_tenant_id = os.getenv("TENANT_ID")
print(f"🔑 Target Tenant ID: {correct_tenant_id}")

# Create a credential chain with tenant-specific authentication
def create_credential_chain_with_tenant():
    """Create a robust credential chain for authentication with specific tenant"""
    try:
        # Try Azure CLI first with the specific tenant
        cli_credential = AzureCliCredential(tenant_id=correct_tenant_id)
        
        # Create a chained credential with fallbacks, all using the correct tenant
        credential_chain = ChainedTokenCredential(
            cli_credential,
            InteractiveBrowserCredential(tenant_id=correct_tenant_id)
        )
        
        return credential_chain
    except Exception as e:
        print(f"⚠️ Credential chain creation error: {e}")
        # Fallback to DefaultAzureCredential with tenant specified
        return DefaultAzureCredential(tenant_id=correct_tenant_id)

# Initialize credentials
try:
    credential = create_credential_chain_with_tenant()
    
    # Test the credential by getting a token for the correct tenant
    test_token = credential.get_token("https://management.azure.com/.default")
    print("✅ Successfully initialized Azure credentials with correct tenant!")
    
    # Double-check tenant by decoding the token (optional debug info)
    import jwt
    try:
        claims = jwt.decode(test_token.token, options={"verify_signature": False})
        actual_tenant = claims.get('tid', 'Unknown')
        print(f"✅ Token tenant: {actual_tenant}")
        if actual_tenant == correct_tenant_id:
            print("✅ Tenant matches - authentication is correctly configured!")
        else:
            print(f"⚠️ Warning: Token tenant {actual_tenant} doesn't match expected {correct_tenant_id}")
    except Exception as decode_error:
        print(f"ℹ️ Could not decode token for verification: {decode_error}")
    
except Exception as e:
    print(f"❌ Credential initialization failed: {str(e)}")
    print(f"\n🔄 The issue is likely that you're authenticated to the wrong tenant.")
    print(f"Expected tenant: {correct_tenant_id}")
    print(f"Please run the authentication fix above to login to the correct tenant.")
    credential = None

### Re-authenticate to Correct Tenant (if needed)

If you're still getting tenant mismatch errors, run this cell to re-authenticate to the correct tenant:

In [ ]:
# Re-authenticate to the correct tenant if needed
correct_tenant_id = os.getenv("TENANT_ID")
subscription_id = os.getenv("AZURE_SUBSCRIPTION_ID")

print(f"🔄 Re-authenticating to correct tenant: {correct_tenant_id}")
print(f"📋 Using subscription: {subscription_id}")

# Force logout and login to correct tenant
!az logout
!az login --use-device-code --tenant {correct_tenant_id}
!az account set --subscription {subscription_id}

# Verify we're in the correct tenant
!az account show --query "{{tenantId: tenantId, subscriptionId: id, name: name}}" --output table

print("\n✅ Re-authentication complete! Try running the credential initialization cell above again.")

## Initialize AI Project Client

> **Note:** Before proceeding, ensure you:
> 1. Copy your `.env.local` file to `.env`
> 2. Update the project connection string in your `.env` file
> 3. Have a Hub and Project already provisioned in Azure AI Foundry

You can find your project connection string in [Azure AI Foundry](https://ai.azure.com) under your project's settings:

<img src="proj-conn-string.png" alt="Project Connection String Location" width="600"/>



## Understanding AIProjectClient

The AIProjectClient is a key component for interacting with Azure AI services that:

- **Manages Connections**: Lists and accesses Azure AI resources like OpenAI models
- **Handles Authentication**: Securely connects using Azure credentials  
- **Enables Model Access**: Provides interfaces to use AI models and deployments
- **Manages Project Settings**: Controls configurations for your Azure AI project

The client requires:
- A project connection string (from Azure AI project settings)
- Valid Azure credentials

You can find your project connection string in Azure AI Studio under Project Settings:



In [ ]:
from dotenv import load_dotenv
from pathlib import Path

# Load environment variables
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir / '.env')

# Initialize AIProjectClient with the correct API
try:
    # Get the project connection string and parse the endpoint
    project_connection_string = os.getenv("PROJECT_CONNECTION_STRING")
    if not project_connection_string:
        raise ValueError("PROJECT_CONNECTION_STRING not found in environment variables")
    
    print(f"🔗 Project Connection: {project_connection_string}")
    
    # Create AIProjectClient using the endpoint URL directly
    client = AIProjectClient(
        endpoint=project_connection_string,
        credential=credential
    )
    print("✓ Successfully initialized AIProjectClient")
except Exception as e:
    print(f"× Error initializing client: {str(e)}")
    print("💡 Tip: Make sure your PROJECT_CONNECTION_STRING is set in the .env file")

## Verify Access to Models and Connections
Let's verify we can access all the required models and connections specified in the [prerequisites](../README.md#-prerequisites).

We need to validate:
- GPT models (gpt-4o, gpt-4o-mini) for chat/completion
- Embedding model for vector search 
- [Grounding with Bing](https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/bing-grounding?view=azure-python-preview&tabs=python&pivots=overview)
- Azure AI Search connection

This validation ensures we have all the components needed to build our AI applications.

In [ ]:
# List the properties of all connections
connections_paged = client.connections.list()
connections = list(connections_paged)  # Convert ItemPaged to list
print(f"====> Listing of all connections (found {len(connections)}):")
for connection in connections:
    # Get connection type - different ways to access it
    conn_type = "Unknown"
    if hasattr(connection, 'type'):
        conn_type = connection.type
    elif hasattr(connection, 'connection_type'):
        conn_type = str(connection.connection_type)
    elif hasattr(connection, '__dict__'):
        # Debug: show available attributes if needed
        conn_type = type(connection).__name__
    
    print(f"  🔗 {connection.name} ({conn_type})")
    
    # Try different ways to get endpoint
    endpoint = None
    if hasattr(connection, 'endpoint'):
        endpoint = connection.endpoint
    elif hasattr(connection, 'endpoint_url'):
        endpoint = connection.endpoint_url
    elif hasattr(connection, 'target'):
        endpoint = connection.target
    
    if endpoint:
        print(f"     Endpoint: {endpoint}")
    print()

# List the properties of all connections of a particular "type" (in this sample, Azure OpenAI connections)
try:
    openai_connections_paged = client.connections.list(
        connection_type=ConnectionType.AZURE_OPEN_AI,
    )
    openai_connections = list(openai_connections_paged)  # Convert to list
    print(f"====> Listing of all Azure Open AI connections (found {len(openai_connections)}):")
    for connection in openai_connections:
        print(f"  🤖 {connection.name}")
        
        # Try to get endpoint
        endpoint = None
        if hasattr(connection, 'endpoint'):
            endpoint = connection.endpoint
        elif hasattr(connection, 'endpoint_url'):
            endpoint = connection.endpoint_url
        elif hasattr(connection, 'target'):
            endpoint = connection.target
            
        if endpoint:
            print(f"     Endpoint: {endpoint}")
        print()
except Exception as openai_error:
    print(f"⚠️ Could not list Azure OpenAI connections: {str(openai_error)}")

# Get the properties of the default connection of a particular "type", with credentials
try:
    ai_services_connection = client.connections.get_default(
        connection_type=ConnectionType.AZURE_AI_SERVICES,
        include_credentials=True,  # Optional. Defaults to "False"
    )
    print("====> Get default Azure AI Services connection:")
    print(f"  🔧 {ai_services_connection.name}")
    
    # Try to get endpoint
    endpoint = None
    if hasattr(ai_services_connection, 'endpoint'):
        endpoint = ai_services_connection.endpoint
    elif hasattr(ai_services_connection, 'endpoint_url'):
        endpoint = ai_services_connection.endpoint_url
    elif hasattr(ai_services_connection, 'target'):
        endpoint = ai_services_connection.target
        
    if endpoint:
        print(f"     Endpoint: {endpoint}")
        
except Exception as e:
    print(f"⚠️ No default Azure AI Services connection found: {str(e)}")

# Show your configured models from environment variables
print("\n====> Your Configured Models (from .env):")
print(f"  🤖 Primary Chat Model: {os.getenv('MODEL_DEPLOYMENT_NAME', 'Not configured')}")
print(f"  📝 Embedding Model: {os.getenv('EMBEDDING_MODEL_DEPLOYMENT_NAME', 'Not configured')}")
print(f"  ⚡ Serverless Model: {os.getenv('SERVERLESS_MODEL_NAME', 'Not configured')}")
print(f"  🔗 Azure OpenAI Deployment: {os.getenv('AZURE_OPENAI_DEPLOYMENT', 'Not configured')}")

# Debug: Show connection object structure for the first connection (if any)
if connections:
    print(f"\n🔍 Debug: First connection object attributes:")
    first_conn = connections[0]
    attrs = [attr for attr in dir(first_conn) if not attr.startswith('_')]
    print(f"   Available attributes: {', '.join(attrs[:10])}{'...' if len(attrs) > 10 else ''}")
    
    # Show key properties
    for attr in ['name', 'type', 'connection_type', 'endpoint', 'endpoint_url', 'target', 'id']:
        if hasattr(first_conn, attr):
            value = getattr(first_conn, attr)
            print(f"   {attr}: {value}")

## Validate Model and Search Connections
The cell below validates that we have properly provisioned and connected to:
1. Azure OpenAI models through our Azure OpenAI connection
2. Azure AI Search through our Azure AI Search connection

Both of these services will be essential for building our AI applications. The OpenAI models will provide the core language capabilities, while Azure AI Search will enable efficient information retrieval and knowledge base functionality.



In [ ]:
# List all connections and check for specific types
conn_list = client.connections.list()
search_conn_id = ""
openai_conn_id = ""

for conn in conn_list:
    # Get connection type using multiple fallback methods
    conn_type = "Unknown"
    if hasattr(conn, 'type'):
        conn_type = str(conn.type).split('.')[-1]
    elif hasattr(conn, 'connection_type'):
        conn_type = str(conn.connection_type).split('.')[-1]
    elif hasattr(conn, '__dict__'):
        # Try to infer from class name or other attributes
        conn_type = type(conn).__name__
    
    # Check for Azure AI Search
    if "AZURE_AI_SEARCH" in conn_type.upper() or "SEARCH" in conn_type.upper():
        search_conn_id = conn.id if hasattr(conn, 'id') else conn.name
    # Check for Azure OpenAI
    elif "AZURE_OPEN_AI" in conn_type.upper() or "OPENAI" in conn_type.upper():
        openai_conn_id = conn.id if hasattr(conn, 'id') else conn.name

print(f"\n====> Connection IDs found:")
if not search_conn_id:
    print("Azure AI Search: Not found - Please create an Azure AI Search connection")
else:
    print(f"Azure AI Search: {search_conn_id}")
    
if not openai_conn_id:
    print("Azure OpenAI: Not found - Please create an Azure OpenAI connection") 
else:
    print(f"Azure OpenAI: {openai_conn_id}")

# Debug: Show what connection types were found
print(f"\n🔍 Debug: Connection types found:")
conn_list_debug = client.connections.list()
for conn in conn_list_debug:
    conn_type = "Unknown"
    if hasattr(conn, 'type'):
        conn_type = str(conn.type)
    elif hasattr(conn, 'connection_type'):
        conn_type = str(conn.connection_type)
    print(f"  - {conn.name}: {conn_type}")